In [ ]:

import cv2
import numpy as np
from keras.models import load_model

# Load the trained model
model_path = "/content/CNN.h5"  # Replace with the actual path to your trained model
loaded_model = load_model(model_path)

# Constants for image dimensions
IMAGE_HEIGHT, IMAGE_WIDTH = 150, 150  # Replace with the dimensions your model expects

# Function to predict class for each frame in a video
def predict_activity(video_path, threshold=0.5, frame_skip=5):
    cap = cv2.VideoCapture(video_path)

    # Initialize variables for accumulating predictions
    accumulated_predictions = 0
    accumulated_frames = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Preprocess the frame
        frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        frame = frame / 255.0  # Normalize pixel values
        frame = np.expand_dims(frame, axis=0)  # Add batch dimension

        # Make prediction
        prediction = loaded_model.predict(frame)

        # Accumulate predictions
        accumulated_predictions += prediction
        accumulated_frames += 1

        # Display the frame with the predicted class every 'frame_skip' frames
        if accumulated_frames % frame_skip == 0:
            # Calculate the average prediction over accumulated frames
            average_prediction = accumulated_predictions / accumulated_frames

            # Display the predicted class if it exceeds the threshold
            if average_prediction > threshold:
                predicted_class = "Violent"
            else:
                predicted_class = "Non-Violent"

            print("Predicted class:", predicted_class)

            # Reset accumulated variables
            accumulated_predictions = 0
            accumulated_frames = 0

        # Break the loop if 'q' key is pressed
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Replace 'video_path' with the path to the video you want to predict
video_path = "/content/Data/Video/Violent/18.mp4"
predict_activity(video_path, threshold=0.7, frame_skip=5)




In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model(export_path_keras)

# Function to preprocess the image for prediction
def preprocess_image(image_path, target_size):
    img = cv2.imread(image_path)
    img = cv2.resize(img, target_size)
    img = img / 255.0  # Normalize the image
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Function to make predictions
def predict_violence(image_path):
    img = preprocess_image(image_path, target_size=(150, 150))
    prediction = loaded_model.predict(img)
    return prediction[0]  # Assuming binary classification, return the prediction for the first class

# Example usage:
image_path = "/content/Data/Training/V/frame_113.jpg"
prediction = predict_violence(image_path)
print("Predicted Probability of Violence:", prediction)
# Function to make predictions and classify as violent or not violent
def predict_violence_class(image_path, threshold=0.5):
    img = preprocess_image(image_path, target_size=(150, 150))
    prediction = loaded_model.predict(img)
    probability = prediction[0]  # Assuming binary classification, get the probability
    if probability >= threshold:
        return "violent", probability
    else:
        return "not violent", probability

# Example usage:
image_path = "/content/Data/Training/V/frame_113.jpg"
class_prediction, probability = predict_violence_class(image_path)
print("Class:", class_prediction)
print("Predicted Probability:", probability)



In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from google.colab.patches import cv2_imshow

# Load the saved model
loaded_model = load_model(export_path_keras)

# Function to preprocess the image for prediction
def preprocess_image(img, target_size):
    img = cv2.resize(img, target_size)
    img = img / 255.0  # Normalize the image
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Function to make predictions and classify as violent or not violent
def predict_violence_class(image, threshold=0.5):
    img = preprocess_image(image, target_size=(150, 150))
    prediction = loaded_model.predict(img)
    probability = prediction[0]  # Assuming binary classification, get the probability
    if probability >= threshold:
        return "violent", probability
    else:
        return "not violent", probability

# Open video capture
cap = cv2.VideoCapture('/content/Data/Video/Violent/23.mp4')

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create VideoWriter object to save the output video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_video.avi', fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Predict violence class for the current frame
    class_prediction, _ = predict_violence_class(frame)

    # Overlay class name on the frame
    cv2.putText(frame, class_prediction, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2_imshow(frame)

    # Write the frame to the output video
    out.write(frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and video writer objects
cap.release()
out.release()
